In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/My Drive/LDS/demo')
!ls

app.py	   keywords.json       __pycache__	 samples.txt	  utils.py
app.pyc    main_sections.json  requirements.txt  samples_val.txt
dancer.py  model	       samples_f1.txt	 server.ipynb


In [3]:
!pip -q install -r requirements.txt

     |████████████████████████████████| 675kB 7.0MB/s 
     |████████████████████████████████| 3.8MB 13.4MB/s 
     |████████████████████████████████| 890kB 41.0MB/s 
     |████████████████████████████████| 1.1MB 57.1MB/s 


In [4]:
from flask_ngrok import run_with_ngrok
from flask import Flask, request
from flask_json import FlaskJSON, json_response
from flask_cors import CORS
from dancer import Dancer, Document
import json
import random

app = Flask(__name__)
app.config['JSON_ADD_STATUS'] = False
flask_json = FlaskJSON()
run_with_ngrok(app)
CORS(app)

samples = [json.loads(article) for article  in open('./samples.txt', 'r')]
n_samples = len(samples)
model = Dancer()

@app.route('/')
def display():
  return 'Display'

@app.route('/summarize', methods=['POST'])
def summarize():
  try:
    document = Document(raw_example=request.json, from_dataset=False)
    summary = model.generate_document_summary(document)
    return json_response(status_=200, summary=summary)
  except Exception as error:
    print('Error:', str(error))
    return json_response(status_=500, error=str(error))
  
@app.route('/random', methods=['GET'])
def random_text():
  idx = random.randint(0, n_samples - 1)
  section_names = samples[idx]['section_names']
  sections = samples[idx]['sections']
  return json_response(status_=200, section_names=section_names, sections=sections)

In [0]:
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://dd727adf0dfa.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [13/Jun/2020 10:24:25] "GET //random HTTP/1.1" 200 -
127.0.0.1 - - [13/Jun/2020 10:24:30] "OPTIONS //summarize HTTP/1.1" 200 -
127.0.0.1 - - [13/Jun/2020 10:24:34] "POST //summarize HTTP/1.1" 200 -


In [5]:
!cat dancer.py

import torch
import json
import utils
from transformers import BartTokenizer, BartForConditionalGeneration

class Document:
   def __init__(self, raw_example, from_dataset=True):
      self.titles = raw_example['section_names']
      if from_dataset:
         self.sections = utils.concate_list(raw_example['sections'])
      else:
         self.sections = raw_example['sections']

class Dancer:
   def __init__(self,
                model_name='facebook/bart-large',
                max_seq_len=1024,
                model_path='./model/dancer.pt',
                device='cuda'):
      self.max_seq_len = max_seq_len
      self.device = device
      self.tokenizer = BartTokenizer.from_pretrained(model_name)
      if model_path is not None:
        self.model = BartForConditionalGeneration.from_pretrained(model_name, state_dict=torch.load(model_path))
      else:
        self.model = BartForConditionalGeneration.from_pretrained(model_name)  
      
      self.model.to(device)      
      self